In [ ]:
from tkinter import *
from PIL import Image as PIL_Image
from PIL import ImageTk
import tkinter.filedialog as tkFileDialog
import cv2
from tkinter import messagebox
import matplotlib.pyplot as plt

In [ ]:
%store -r result_list         
%store -r path                

In [ ]:
def display_output():
    sub_frame1 = Frame(master_frame, bg='black', pady=5)
    sub_frame2 = Frame(master_frame, bg='brown', pady=5, padx = 5)
    sub_frame3 = Frame(master_frame, pady=5, padx = 5)
    sub_frame4 = Frame(master_frame, pady=5, padx = 5)

    sub_frame1.grid(row = 1, column = 1, padx = 5, pady = 5)
    sub_frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
    sub_frame3.grid(row = 3, column = 1, padx = 5, pady = 5)
    sub_frame4.grid(row = 4, column = 1, padx = 5, pady = 5)

    title = Label(master=sub_frame1, text="FAKE CURRENCY DETECTION SYSTEM", fg = 'dark blue', font = "Verdana 28 bold")
    title.pack()

    canvas_input = Canvas(master=sub_frame2, width = 675, height = 300)  
    canvas_input.pack()

    if len(path) > 0 and path[-4:] == '.jpg':
        image = cv2.imread(path)
        original_image = image.copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (675, 300))
        image = PIL_Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        canvas_input.image = image
        canvas_input.create_image(0, 0, anchor=NW, image=image) 
    
    pass_count = 0

    for i in range(4):
        for j in range(3):
            feature_num = 3*i+j

            if feature_num < 10:
                sub_frame4.grid_rowconfigure(i, weight=1)
                sub_frame4.grid_columnconfigure(j, weight=1)

                feature_frame = Frame(master = sub_frame4, relief = RAISED, borderwidth = 1, bg='light blue')
                feature_frame.grid(row = i, column = j, padx = 20, pady = 20, sticky="nsew")

                frame1 = Frame(feature_frame, padx=3, pady=3)
                frame2 = Frame(feature_frame, bg='brown', pady=5, padx = 5)
                frame3 = Frame(feature_frame)
                frame4 = Frame(feature_frame)
                frame5 = Frame(feature_frame)

                frame1.grid(row = 1, column = 1, padx = 5, pady = 5, ipadx = 100)
                frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
                frame3.grid(row = 3, column = 1, padx = 5, pady = 5)
                frame4.grid(row = 4, column = 1, padx = 5, pady = 5)
                frame5.grid(row = 5, column = 1, padx = 5, pady = 5)

                label1 = Label(master = frame1, text = f"Feature {feature_num +1}", fg = 'black', font = "Verdana 12 bold")
                label1.pack()

                canvas = Canvas(master=frame2, width = 200, height = 200)  
                canvas.pack()

                image = result_list[feature_num][0].copy()

                h, w = image.shape[:2]
                aspect_ratio = w/h

                resize_height = 0
                resize_width = 0
                img_x = 0
                img_y = 0
                
                if h > w:
                    resize_height = 200
                    resize_width = aspect_ratio * resize_height
                    img_x = (200 - resize_width)/2
                elif h < w:
                    resize_width = 200
                    resize_height = resize_width / aspect_ratio
                    img_y = (200 - resize_height)/2
                else:
                    resize_height = 200
                    resize_width = 200
                
                resize_height = int(resize_height)
                resize_width = int(resize_width)
                img_x = int(img_x)
                img_y = int(img_y)

                image = cv2.resize(image, (resize_width, resize_height))

                image = PIL_Image.fromarray(image)
                image = ImageTk.PhotoImage(image)

                canvas.image = image
                canvas.create_image(img_x, img_y, anchor=NW, image=image) 

                if feature_num < 7:
                    avg_score = result_list[feature_num][1]
                    avg_score = "{:.3f}".format(avg_score)
                    text2 = "Avg. SSIM Score: " + avg_score
                elif feature_num < 9:
                    line_count = result_list[feature_num][1]
                    line_count = "{:.3f}".format(line_count)
                    text2 = "Avg. Number of lines: " + line_count
                else:
                    status = result_list[feature_num][1]
                    if status == True:
                        text2 = "9 characters detected!"
                    else:
                        text2 = "Less than 9 characters detected!"
                label2 = Label(master = frame3, text = text2, fg = 'dark blue', font = "Verdana 11", bg='light blue')
                label2.pack()

                if feature_num < 7:
                    max_score = result_list[feature_num][2]
                    max_score = "{:.3f}".format(max_score)
                    text3 = "Max. SSIM Score: " + max_score
                elif feature_num < 9:
                    text3 = ""
                else:
                    text3 = ""
                label3 = Label(master = frame4, text = text3, fg = 'dark blue', font = "Verdana 11", bg='light blue')
                label3.pack()

                if feature_num < 7:
                    status = result_list[feature_num][3]
                elif feature_num < 9:
                    status = result_list[feature_num][2]
                else:
                    status = result_list[feature_num][1]
                
                if status == True:
                    pass_count += 1
                    label4 = Label(master = frame5, text = "Status: PASS!", fg = 'green', font = "Verdana 11 bold", bg='light blue')
                    label4.pack()
                else:
                    label4 = Label(master = frame5, text = "Status: FAIL!", fg = 'red', font = "Verdana 11 bold", bg='light blue')
                    label4.pack()

    result = Label(master=sub_frame3, text= f"RESULT: {pass_count} / 10 features PASSED!", fg = 'green', font = "Verdana 24 bold")
    result.pack()


def scrollbar_function(event):
    canvas.configure(scrollregion=canvas.bbox("all"),width=1050,height=550)


root=Tk()
root.title('Fake Currency Detection - Result Analysis')

root.resizable(False, False)

window_height = 600
window_width = 1100

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

x_cordinate = int((screen_width/2) - (window_width/2))
y_cordinate = int((screen_height/2) - (window_height/2))

root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

main_frame=Frame(root,relief=GROOVE, bd=1)
main_frame.place(x=10,y=10)

canvas=Canvas(main_frame)
master_frame=Frame(canvas)

myscrollbar=Scrollbar(main_frame,orient="vertical",command=canvas.yview)
canvas.configure(yscrollcommand=myscrollbar.set)

myscrollbar.pack(side="right",fill="y")
canvas.pack(side="left")
canvas.create_window((0,0),window=master_frame,anchor='nw')
master_frame.bind("<Configure>",scrollbar_function)

display_output()

root.mainloop()